In [14]:
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image, ImageDraw

In [1]:
%run keymagic.py

In [5]:
kbrd = keyboardIOS7()
n_chars = len(kbrd)
char_indices = dict((c, i) for i, c in enumerate(kbrd.keys()))
indices_char = dict((i, c) for i, c in enumerate(kbrd.keys()))

In [17]:
with Image.open(keyboard_img) as im:
    im_size_x, im_size_y = im.size

with open('tiny-shakespeare.txt') as f:
    text = f.read().lower()
    
clean_text = re.sub("[^a-zA-Z'-]", ' ', text).lower()

In [15]:
n_points = 100
max_letters = 20

In [324]:
%%time
X, y = generate_dataset_words(clean_text, n_points=n_points, max_letters=max_letters)

/Users/dmitri/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


CPU times: user 40min 36s, sys: 1min 10s, total: 41min 47s
Wall time: 44min 17s


In [325]:
import h5py
import os

with h5py.File(os.path.expanduser('~/temp/keymagic-train-shakespeare.h5'), 'w') as f:
    f.create_dataset('X', data=X, compression="gzip")
    f.create_dataset('y', data=y, compression="gzip")

In [185]:
with h5py.File(os.path.expanduser('~/temp/keymagic-train-shakespeare.h5')) as f:
    X = f['X'][:]
    y = f['y'][:]

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, RepeatVector, TimeDistributed
from keras.layers import LSTM
from keras.layers import Convolution1D, MaxPooling1D
from keras.optimizers import RMSprop

model = Sequential()
model.add(Convolution1D(8, 4, input_shape=(n_points, 3)))
model.add(Activation('relu'))
model.add(Convolution1D(8, 4))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_length=5))
model.add(Convolution1D(1, 1))
model.add(Flatten())
model.add(RepeatVector(max_letters))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars)))
model.add(TimeDistributed(Activation('softmax')))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_16 (Convolution1D) (None, 97, 8)         104         convolution1d_input_6[0][0]      
____________________________________________________________________________________________________
activation_12 (Activation)       (None, 97, 8)         0           convolution1d_16[0][0]           
____________________________________________________________________________________________________
convolution1d_17 (Convolution1D) (None, 94, 8)         264         activation_12[0][0]              
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 94, 8)         0           convolution1d_17[0][0]           
___________________________________________________________________________________________

In [327]:
model.fit(X, y, batch_size=256, nb_epoch=10)

Epoch 1/10
194146/194146 [==============================] - 133s - loss: 0.6360   
Epoch 2/10
194146/194146 [==============================] - 132s - loss: 0.4796   
Epoch 3/10
194146/194146 [==============================] - 128s - loss: 0.4482   
Epoch 4/10
194146/194146 [==============================] - 145s - loss: 0.4326   
Epoch 5/10
194146/194146 [==============================] - 141s - loss: 0.4254   
Epoch 6/10
194146/194146 [==============================] - 151s - loss: 0.4173   
Epoch 7/10
194146/194146 [==============================] - 150s - loss: 0.4107   
Epoch 8/10
194146/194146 [==============================] - 153s - loss: 0.6307   
Epoch 9/10
194146/194146 [==============================] - 164s - loss: 0.5571   
Epoch 10/10
194146/194146 [==============================] - 136s - loss: 0.4074   


In [337]:
l = generate_input('man', n_points)
p = model.predict(np.array([l]))
''.join([indices_char[ix] for ix in np.argmax(p[0], axis=1)]).strip()

/Users/dmitri/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


'man'

In [361]:
l = generate_input('say', n_points)
p = model.predict(np.array([l]))
''.join([indices_char[ix] for ix in np.argmax(p[0], axis=1)]).strip()

/Users/dmitri/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


'aay'

In [371]:
l = generate_input('citizen', n_points)
p = model.predict(np.array([l]))
''.join([indices_char[ix] for ix in np.argmax(p[0], axis=1)]).strip()

/Users/dmitri/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


'cooiiann'

In [349]:
model.save('models/keymagic-ts-10-epoch.hdf')

In [350]:
from keras.models import load_model

model = load_model('models/keymagic-ts-10-epoch.hdf')